## Problem 1: Generating Random Boolean Functions

The idea for this problem is to randomly generate either a constant value (True or False) for all possible combinations of n binary inputs, or return True (which are randomly distributed) for exactly half of the combinations (and False for the other half).

For n inputs there will be 2^n outputs as there are 2^n combinations.

In [4]:
# Required imports
import random
from itertools import product

In [5]:
def random_constant_balanced(num_inputs):
    table_size = 2 ** num_inputs # This will always be even, so we do not have to worry about uneven True and False counts.
    
    #
    if random.choice(['constant', 'balanced']) == 'constant':
        return np.full(table_size, random.choice(['0', '1']))
    else:
        half = table_size // 2
        result = np.array(['0'] * half + ['1'] * half)
        np.random.shuffle(result)
        return result